Consiste en un ensamble de los modelos 5, 6 y 7 que individualmente tienen un rmse: 0.0785   0.0747  0.07965

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer

In [2]:
data=pd.read_csv("../data/datos_modificados.csv")
data=data.drop(columns="id")

In [33]:
data_usable=data[data["x_e_out [-]"].notnull()].copy()
prediction=data[data["x_e_out [-]"].isnull()].copy()

In [34]:
# Divido los datos en características (X) y columna objetivo (y)
X = data_usable.drop('x_e_out [-]', axis=1)
y = data_usable['x_e_out [-]']

# Identificar las columnas categóricas
categorical_cols = ['author', 'geometry']

# Definir el transformador para la codificación one-hot
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')

# Aplicar la codificación one-hot a las variables categóricas
X_encoded = ct.fit_transform(X)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, shuffle=True)

In [36]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Modelo 5

In [37]:
model5 = DecisionTreeRegressor(random_state=42,max_depth= 9, min_samples_leaf=6, min_samples_split= 20)
model5.fit(X_train_scaled, y_train)
y_pred5 = model5.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred5))
rmse

0.07850422278685386

Modelo 6

In [38]:
model6 = RandomForestRegressor(max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=10, n_estimators=200)
model6.fit(X_train_scaled, y_train)
y_pred6 = model6.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred6))
rmse

0.07475886473322982

Modelo 7

In [39]:
model7 = SVR(C= 1, epsilon= 0.1, gamma= 'scale', kernel= 'rbf')
model7.fit(X_train_scaled, y_train)
y_pred7 = model7.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred7))
rmse

0.07965810821588803

Ensamble 1

In [40]:
from sklearn.linear_model import LinearRegression

In [41]:
from sklearn.ensemble import  StackingRegressor

meta_model = LinearRegression()

# Crea el apilamiento de regresión
stacking_regressor = StackingRegressor(estimators=[('model1', model5), ('model2', model6), ('model3', model7)], final_estimator=meta_model)
stacking_regressor.fit(X_train_scaled, y_train)

# Realiza predicciones con el apilamiento
y_pred_regression = stacking_regressor.predict(X_test_scaled)


In [42]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_regression))
rmse

0.07469094532448543

In [43]:
prediction=prediction.drop(columns="x_e_out [-]")

In [46]:
prediction_encoded = ct.fit_transform(prediction)
prediction_final=scaler.fit_transform(prediction_encoded)

In [48]:
submission_values=stacking_regressor.predict(prediction_final)

In [51]:
index_submission=prediction.index

In [54]:
df = pd.DataFrame({'id': index_submission, 'x_e_out [-]': submission_values})
df.to_csv("submissions/Submission2_model8.csv", index=False)
df

,id,x_e_out [-]
0,4,-0.015526
1,7,-0.080263
2,10,-0.051152
3,12,0.014763
4,23,0.055743
...,...,...
10410,31633,0.088688
10411,31634,-0.045809
10412,31637,0.037908
10413,31640,-0.050325


Ensamble 2

In [55]:
from sklearn.ensemble import VotingRegressor


# Crea el ensamble de regresión
voting_regressor = VotingRegressor(estimators=[('model1', model5), ('model2', model6), ('model3', model7)])
voting_regressor.fit(X_train_scaled, y_train)

# Realiza predicciones con el ensamble

y_pred_regression = voting_regressor.predict(X_test_scaled)


In [56]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_regression))
rmse

0.07554328359414704

Ensamble 3

In [57]:
y_pred_model1 = model5.predict(X_test_scaled)
y_pred_model2 = model6.predict(X_test_scaled)
y_pred_model3 = model7.predict(X_test_scaled)

# Calcula el promedio de las predicciones
y_pred_ensemble = (y_pred_model1 + y_pred_model2 + y_pred_model3) / 3

In [58]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
rmse

0.07546943009804515

Ensamble 4

In [59]:
from sklearn.tree import DecisionTreeRegressor

In [62]:
model = DecisionTreeRegressor(random_state=562)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [63]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

0.10668303624783625

In [79]:
from sklearn.ensemble import  StackingRegressor

meta_model = LinearRegression()

# Crea el apilamiento de regresión
stacking_regressor = StackingRegressor(estimators=[('model1', model5), ('model2', model6), ('model3', model7),('model4', model)], final_estimator=meta_model)
stacking_regressor.fit(X_train_scaled, y_train)

# Realiza predicciones con el apilamiento
y_pred_regression = stacking_regressor.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_regression))
print(rmse)

submit1=stacking_regressor.predict(prediction_final)

0.0745640921559147


In [80]:
from sklearn.ensemble import  StackingRegressor

meta_model = RandomForestRegressor()

# Crea el apilamiento de regresión
stacking_regressor = StackingRegressor(estimators=[('model1', model5), ('model2', model6), ('model3', model7),('model4', model)], final_estimator=meta_model)
stacking_regressor.fit(X_train_scaled, y_train)

# Realiza predicciones con el apilamiento
y_pred_regression2 = stacking_regressor.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_regression))
print(rmse)
submit2=stacking_regressor.predict(prediction_final)

0.0745640921559147


In [81]:
from sklearn.ensemble import  GradientBoostingRegressor

meta_model = GradientBoostingRegressor()

# Crea el apilamiento de regresión
stacking_regressor = StackingRegressor(estimators=[('model1', model5), ('model2', model6), ('model3', model7),('model4', model)], final_estimator=meta_model)
stacking_regressor.fit(X_train_scaled, y_train)

# Realiza predicciones con el apilamiento
y_pred_regression = stacking_regressor.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_regression))
print(rmse)
submit3=stacking_regressor.predict(prediction_final)

0.0745267882866566


In [77]:
union3=((submit1+submit2+submit3)/3)

In [82]:
df = pd.DataFrame({'id': index_submission, 'x_e_out [-]': submit3})
df.to_csv("submissions/Submission4_model8.csv", index=False)
df

,id,x_e_out [-]
0,4,-0.016493
1,7,-0.071896
2,10,-0.057653
3,12,0.017860
4,23,0.056318
...,...,...
10410,31633,0.081017
10411,31634,-0.043822
10412,31637,0.037059
10413,31640,-0.047986


In [74]:
submit1[0]

-0.008324327406787645

In [75]:
submit2[0]

-0.021125000000000015

In [76]:
submit3[0]

-0.012326333263975673